In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA

In [ ]:
path = '/content/drive/MyDrive/Data Science/Project-39 Clothing Fit-Size predictions/cloth_yelp.json'
df= pd.read_json(path, lines=True)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.drop(['item_id', 'user_id', 'review_summary', 'review_text', 'user_name'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

### filling waist null values

In [ ]:
waist_cor = df.groupby('size')['waist'].mean()
waist_cor

In [ ]:
def fillWaist():
  x = []
  for i in range(len(df['waist'])):
    if math.isnan(df['waist'][i]):
      x.append(round(waist_cor[df['size'][i]], 1))
    else:
      x.append(df['waist'][i])
  return x

In [ ]:
df['waist'] = fillWaist()

In [ ]:
df.head()

### Height

In [ ]:
df['height'] = df['height'].str.replace('ft', '.').str.replace('in', '').str.replace(' ', '')

In [ ]:
df['height'] = df['height'].apply(lambda x: np.float64(x) if x != 'NaN' or x != 'nan' else 'NaN')

### Bust

In [ ]:
df['bust'] = df['bust'].replace('37-39', '38').apply(lambda x: np.float64(x))

In [ ]:
bust = df.groupby('size')['bust'].median()
bust

In [ ]:
def fill_bust():
  x = []
  for i in range(len(df['bust'])):
    if math.isnan(df['bust'][i]):
      x.append(round(bust[df['size'][i]], 1))
    else:
      x.append(df['waist'][i])
  return x

In [ ]:
df['bust'] = fill_bust()

### dropping Null values

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
obj_col = df.select_dtypes('object').columns

In [ ]:
for i in obj_col:
  print(i)
  print(df[i].value_counts())
  print('---'*20)

In [ ]:
for i in obj_col:
  encoder = LabelEncoder().fit(df[i])
  df[i] = encoder.transform(df[i])
  print(i)
  print(dict(zip(encoder.classes_, encoder.transform(encoder.classes_))))
  print('----'*20)

In [ ]:
df.head()

## Checking outlayers and corration

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(13, 13))
sns.heatmap(df.corr(), annot=True, cmap='viridis', linewidths=0.8)
plt.show()

In [ ]:
sns.boxplot(data=df, x='waist')

In [ ]:
sns.boxplot(data=df, x='size')

In [ ]:
sns.boxplot(data=df, x='hips')

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
x = df.drop(['waist', 'quality', 'hips'], axis=1)
y = df['quality'].values

In [ ]:
scaler = StandardScaler().fit(x)
x = scaler.transform(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=111)

In [ ]:
# import models
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
all_model = [LogisticRegression, RandomForestClassifier, DecisionTreeClassifier]

In [ ]:
def getModel_score(x):
  model = x()
  model.fit(x_train, y_train)
  print(type(model).__name__)
  print(model.score(x_train, y_train))
  print(model.score(x_test, y_test))


In [ ]:
for i in all_model:
  getModel_score(i)